In [1]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join("..")))

In [2]:
from filter_nutzungsflaechen import (
    get_crops_to_exclude,
    get_rainfed_reference_crops,
    create_crop_filters,
    filter_crops,
    add_double_cropping_info,
)

from compute_ET_green import compute_et_green
from compute_ET_blue import compute_et_blue, compute_volumetric_et_blue

from utils import ee_utils

import ee
import geemap

In [ ]:
ee.Initialize(project="thurgau-irrigation")

## Define the region of interest

In [ ]:
# For the canton of Thurgau
cantonal_borders_asset = (
    "projects/thurgau-irrigation/assets/Thurgau/thrugau_borders_2024"
)

aoi_feature_collection = ee.FeatureCollection(cantonal_borders_asset)
aoi_geometry = aoi_feature_collection.geometry()
aoi_geometry = aoi_geometry.simplify(500)
aoi_buffered = aoi_geometry.buffer(100)

## Importing all assets: land use maps, vegetation periods, WaPOR ET data.

In [ ]:
years_to_process = range(2018, 2023)

double_cropping_image_collection = ee.ImageCollection(
    "projects/thurgau-irrigation/assets/Thurgau/vegetation_periods_2018-2023"
)


WaPOR_assets = "projects/thurgau-irrigation/assets/Thurgau/ET_WaPOR_10m_dekadal"
WaPOR_et_collections = ee_utils.merge_collections(
    years=years_to_process, asset_name=WaPOR_assets
)


def back_to_float(image: ee.Image, scale: int) -> ee.Image:
    """
    Convert an image to float and divide by the scale

    Args:
        image: The image to convert
        scale: The scale to divide by

    Returns:
        The image converted to float and divided by the scale
    """
    date = image.get("system:time_start")
    return image.toFloat().divide(scale).set("system:time_start", date)


WaPOR_et_collections = WaPOR_et_collections.map(lambda image: back_to_float(image, 100))

## Filtering out potentially irrigated crops and refeence rainfed fields

In [ ]:
# # Add double cropping information to the feature collection
# nutzung_collection_with_double_crop = add_double_cropping_info(
#     nutzung_collection, double_cropping_image_2018
# )

# not_irrigated_crops = get_crops_to_exclude()
# rainfed_crops = get_rainfed_reference_crops()

# exclude_filter, rainfed_filter = create_crop_filters(not_irrigated_crops, rainfed_crops)

# potentially_rainfed, rainfed_fields = filter_crops(
#     nutzung_collection_with_double_crop, exclude_filter, rainfed_filter
# )

# print("Filtered fields count:", potentially_rainfed.size().getInfo())
# print("Rainfed reference fields count:", rainfed_fields.size().getInfo())

## Computing ET green and exporting the results

In [ ]:
# et_image = WaPOR_et_collections.first()

# et_green_result = compute_et_green(et_image, rainfed_fields)

In [ ]:
def export_image_to_asset(
    image: ee.Image,
    asset_id: str,
    task_name: str,
    year: int,
    aoi: ee.Geometry,
    max_pixels: int = 1e13,
) -> ee.batch.Task:
    """
    Export an image to an Earth Engine asset.
    """
    task = ee.batch.Export.image.toAsset(
        image=image,
        description=task_name,
        assetId=asset_id,
        region=aoi,
        scale=10,
        maxPixels=max_pixels,
    )
    print(f"Exporting {task_name} for {year} to {asset_id}")
    task.start()
    return task


double_cropping_list = double_cropping_image_collection.toList(
    double_cropping_image_collection.size()
)
WaPOR_et_collections_list = WaPOR_et_collections.toList(WaPOR_et_collections.size())

not_irrigated_crops = get_crops_to_exclude()
rainfed_crops = get_rainfed_reference_crops()

exclude_filter, rainfed_filter = create_crop_filters(not_irrigated_crops, rainfed_crops)

# tasks = []

# total_tasks = 0

# for year in years_to_process:
#     j = year - 2018
#     nutzung_collection_year = ee.FeatureCollection(
#         f"projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_{year}_area"
#     )
#     double_cropping_image_year = ee.Image(double_cropping_list.get(j))

#     nutzung_collection_with_double_crop_year = add_double_cropping_info(
#         nutzung_collection_year, double_cropping_image_year
#     )

#     potentially_rainfed_year, rainfed_fields_year = filter_crops(
#         nutzung_collection_with_double_crop_year, exclude_filter, rainfed_filter
#     )

#     year_tasks = 0

#     for i in range(36):
#         et_image_year = ee.Image(WaPOR_et_collections_list.get(j * 36 + i))
#         et_green_result_year = compute_et_green(et_image_year, rainfed_fields_year)

#         dekadal = i % 3 + 1
#         month = i // 3 + 1

#         task_name = f"ET_green_{year}-{month:02d}_D{dekadal}"
#         asset_id = (
#             f"projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022/{task_name}"
#         )

#         task = export_image_to_asset(
#             et_green_result_year, asset_id, task_name, year, aoi_buffered
#         )
#         tasks.append(task)
#         year_tasks += 1
#         total_tasks += 1

#     print(f"Year {year} processing complete. Started {year_tasks} tasks.")

# print(f"All processing complete. Total tasks started: {total_tasks}")

## Testing that it all went fine

In [ ]:
# tasks = []

# total_tasks = 0

# et_green_collection = ee.ImageCollection(
#     "projects/thurgau-irrigation/assets/Thurgau/ET_green_2018-2022"
# ).map(lambda image: back_to_float(image, 10000))

# et_green_collection_list = et_green_collection.toList(et_green_collection.size())

# for year in years_to_process:
#     j = year - 2018

#     year_tasks = 0

#     for i in range(36):
#         et_image_year = ee.Image(WaPOR_et_collections_list.get(j * 36 + i))
#         et_green_year = ee.Image(et_green_collection_list.get(j * 36 + i))

#         et_blue_mm = compute_et_blue(et_image_year, et_green_year)
#         et_blue_m3 = compute_volumetric_et_blue(et_blue_mm)

#         et_blue = et_blue_mm.addBands(et_blue_m3)

#         date = et_image_year.get("system:time_start")

#         # Multiply by 100 and cast to int to save space
#         et_blue = et_blue.multiply(100).int().set("system:time_start", date)

#         dekadal = i % 3 + 1
#         month = i // 3 + 1

#         task_name = f"ET_blue_{year}-{month:02d}_D{dekadal}"
#         asset_id = (
#             f"projects/thurgau-irrigation/assets/Thurgau/ET_blue_2018-2022/{task_name}"
#         )

#         task = export_image_to_asset(et_blue, asset_id, task_name, year, aoi_buffered)
#         tasks.append(task)
#         year_tasks += 1
#         total_tasks += 1

#     print(f"Year {year} processing complete. Started {year_tasks} tasks.")

# print(f"All processing complete. Total tasks started: {total_tasks}")

## Exploring Nutzungsflaechen

In [10]:
nutzung_collection = ee.FeatureCollection(
    "projects/thurgau-irrigation/assets/Thurgau/Nutzungsflaechen/TG_2018_area"
)

In [11]:
def get_unique_nutzung_values(feature_collection: ee.FeatureCollection) -> list:
    """
    Get all unique values of the 'nutzung' property from a feature collection.

    Args:
        feature_collection (ee.FeatureCollection): The input feature collection.

    Returns:
        list: A list of unique 'nutzung' values.
    """
    # Get distinct values of 'nutzung'
    distinct_values = feature_collection.distinct('nutzung')

    # Aggregate the distinct values into a list
    unique_values = distinct_values.aggregate_array('nutzung')

    # Get the result as a Python list
    return unique_values.getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[8.727682593683012, 47.65023476440003],
    [8.727972354263729, 47.65023924800176],
    [8.728810690294116, 47.65025710257947],
    [8.730014695233493, 47.65027937624826],
    [8.730522948424099, 47.65029718424333],
    [8.731726922055417, 47.650355151535706],
    [8.731735901804308, 47.65038639129922],
    [8.73138356470141, 47.65036854473621],
    [8.730638920984049, 47.6503284145219],
    [8.729805108044017, 47.65030614205983],
    [8.72835137714512, 47.65028827741762],
    [8.727731571346789, 47.65027936414204],
    [8.727682593683012, 47.65023476440003]]]},
 'id': '000000000000000001a9',
 'properties': {'Area': 1060.028,
  'anzahl_bae': None,
  'beitragsbe': 1,
  'bewirtsc_1': 'f37d74c4-3eff-49b8-ac91-08ed6aaa0902',
  'bewirtscha': 100,
  'datum_bis': '2019/12/12 00:00:00.000',
  'datum_von': '2018/12/12 00:00:00.000',
  'einzelkult': 0,
  'hid': 116244,
  'ist_aggreg': 0,
  'ist_defini': 1,
  'ist_geomet': 1,